In [30]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import datetime
import json

# LangChain for LLM Application Development
# Models, Prompt and parser 파트

# 개요: 영어가 아닌 다른 언어로 메일을 수신하게 된 상황을 가정, 영어 해적어로 메일 수신
# 수신된 내용을 정중하고 친절한 영어로 변환

# 라인 체인 추상화에 대한 내용
# API 키와 OpenAI 모듈 필요

# gpt/api key: 현재 미발급
OPEN_API_KEY = "OPEN_API_KEY"

# dotenv 라이브러리를 활용해 .env 파일을 로드한다.
# .env 파일의 경로를 명시적으로 지정한다.
dotenv_path = 'LLM.env'
print(f"Dotenv path: {dotenv_path}")  # 디버깅 코드 추가
_ = load_dotenv(dotenv_path)

# os.environ은 환경 변수를 딕셔너리 형태로 저장하는 객체
# .env 파일에 저장된 실제 API 키 값을 가져온다.
if OPEN_API_KEY in os.environ:
    api_key = os.environ[OPEN_API_KEY]
    print(f"API Key loaded: {api_key}")  # 디버깅 코드 추가
else:
    raise KeyError(f"{OPEN_API_KEY} not found in environment variables.")

# OpenAI 클라이언트 생성
client = OpenAI(api_key=api_key)

# 현재 날짜에 따라 GPT 모델을 변경
# LLM 모델은 항상 동일한 결과를 도출하지 않기 때문에 예제와는 다른 결과를 얻을 수 있음
current_date = datetime.datetime.now().date()
target_date = datetime.date(2024, 7, 15)

llm_model = "gpt-3.5-turbo"

def get_completion(prompt, model=llm_model):
    response = client.completions.create(
        model=model,
        prompt=prompt,
        max_tokens=100,
        temperature=0
    )
    return response.choices[0].text.strip()

# 구매자로부터 온 email의 원문
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# 번역 스타일 요청
style = "차분하고 존중하는 어조의 미국 영어"

# 명령 프롬프트 설정
prompt = f"""아래의 세 개의 백틱으로 구분된 텍스트를 {style} 스타일로 번역하세요.
text: ```{customer_email}```
"""

response = get_completion(prompt)
print(response)

# 추가된 코드 (before -> after 변환)
completion = client.completions.create(model='curie', prompt="Translate the text.")
print(completion.choices[0].text)
print(dict(completion).get('usage'))
print(json.dumps(completion, indent=2))


Dotenv path: LLM.env
API Key loaded: sk-proj-Y3tCO1Ynj9OzhEJQz5RaT3BlbkFJvSPhGNXV4w8MIHoPEnOT


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
!pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.35.13
    Uninstalling openai-1.35.13:
      Successfully uninstalled openai-1.35.13


In [31]:
# langchain_community 패키지 필요
# f-string 이 아닌 프롬프트 템플릿을 권장하는 이유는 어플리케이션이 거대해 질수록
# 프롬프트를 재사용할 일이 생기기 때문이다.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 온도가 0.0인 3.5 turbo model 생성
chat = ChatOpenAI(temperature=0.0, model=llm_model)

# 템플릿 형태의 프롬프트 디자인 형식
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# ChatPromptTemplate 패키지 하위의 from_template() 함수를 이용해 작성한 template_string 주입
prompt_template = ChatPromptTemplate.from_template(template_string)

# 첫번째 메시지에 대한 프롬프트로 message는 여러 메시지를 포함한다. .prompt는 해당 메시지의 프롬프트 내용을 뜻한다.
prompt_template.messages[0].prompt

# 첫번째 프롬프트에 사용된 변수들을 뜻한다. 현재 템플릿에 의하면 style과 text 를 뜻한다.
prompt_template.messages[0].prompt.input_variables

# 원하는 스타일 지정
customer_style = """American English \
in a calm and respectful tone
"""

# 실제로 수신한 메일 내용
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# 결과를 customer_messages에 대입
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))


service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""




ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)